In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics 
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

In [2]:
importlib.reload(f)

<module 'scripts.functions' from '../scripts/functions.py'>

In [3]:
from conf import *
from numba import njit, jit

In [4]:
# P = np.load(parentdir+'/save/dynamics/h=1.0V=0.9l=[4, 4]/P_n=1.0e+04_1.npy')
# T = np.load(parentdir+'/save/dynamics/h=1.0V=0.9l=[12, 12]/T_n=1.0e+04_0.npy')

In [ ]:
length = [20, 20]
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))


l = []
for i in range(length[0]):
    for j in range(length[1]):
        l.append([i, j])
l = np.array(l)
a = 0
a_ = [a for _ in range(np.prod(length))]
a = np.array(a_)



edges, colors = hex_.dimer_corr(l,a)
operators = f.return_spin_corr(hi, edges, colors)


In [8]:
@njit
def cal_mels(P, T, t_list):
    mels_list = np.zeros((t_list.shape[0],)+P.shape[1:] , dtype=P.dtype)
#     num_samples = np.zeros_like(T, dtype=nb.boolean)
#     sections = np.zeros(t_list.shape[0])
    t_list_ = t_list.reshape(-1,1)

    S = 0
    for n in range(T.shape[0]-1):
        
#         A = T <= t_list[n]
#         num_samples = np.logical_or(A, )
#         ind1, ind2 = np.where(np.logical_xor(A[1:,:],A[:-1,:]))
#     #     sections[n] = S
#         n_state = ind1.shape[0]

        index = np.logical_and(T[n] <= t_list_,  t_list_ < T[n+1])

        for i in range(index.shape[1]):
            mels_list[index[:,i],i] = P[n,i]
    return mels_list

In [6]:
import numpy as np
a = np.tan((1/6)*np.pi) * 1/2
edge_coors_prime = np.array([
    hex_.a((1/2)*np.pi) * a,
    hex_.a(np.pi*(2/3)+(1/2)*np.pi) * a,
    hex_.a(np.pi*(4/3)+(1/2)*np.pi) * a,
])

edge_coors = np.array([[ 0.5     ,  0.      ],
                       [ 0.25    , -0.433013],
                       [ 0.75    , -0.433013]])

base_edge = edge_coors[0]
x = np.arange(length[0]*2)
y = np.arange(length[1]*2)

xx, yy = np.meshgrid(x,y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)
edge_coor_array = hex_.ProcessPeriodic(xx[:,None] * hex_.a1[None,:] * 1/2 + yy[:,None]  * hex_.a2[None,:] * 1/2 + base_edge)
edges = hex_.edge_coor_to_lattice(edge_coor_array)
colors = hex_.get_edge_color(edges)

operators = f.return_spin_corr(hi, edges, colors)

In [7]:
hex_.b1 / 2

array([0.5       , 0.28867514])

In [8]:
from plotly import graph_objects as go
data1 = go.Scatter(
    x = hex_.lattice_coor[:,0],
    y = hex_.lattice_coor[:,1],
    text=[f'{i}' for i in range(np.prod(hex_.l) * 2)],
    mode = 'markers+text',
    textposition="top center"
)

data2 = go.Scatter(
    x = edge_coor_array[:,0],
    y = edge_coor_array[:,1],
    mode = 'markers',
)

fig = go.Figure()
fig.add_traces(
    [data1, data2]
)
fig.layout.yaxis = {
    'scaleanchor': 'x', 'scaleratio': 1
}
fig.layout.yaxis.showgrid = False
fig.layout.xaxis.showgrid = False
fig.write_image("fig1.jpeg")

In [86]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])


_, mels1 = operators[0].get_conn_flattened(P_list_, sections2)
mels1 = mels1.reshape(P_list.shape[0], P_list.shape[1]).real
sub1 = mels1[0].mean().real

size = len(operators)

dimer_corr = np.zeros((size,t_list.shape[0]))
dimer_std = np.zeros((size,t_list.shape[0]))


for s in range(size):
    print(s)
    if operators[s]:

        _, mels2 = operators[s].get_conn_flattened(P_list[0,:,:], sections1)
        mels2 = mels2.real
        sub2 = mels2.mean().real
#         mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        dimer_corr[s] = (mels2 * mels1).mean(axis=1)
        dimer_std[s] = (mels2 * mels1).std(axis=1)
    else:
        dimer_corr[s] = 0
        dimer_std[s] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [2]:
import numpy as np
X = np.ma.masked_equal(np.random.rand(100),0)

In [39]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])


_, mels1 = operators[0].get_conn_flattened(P_list[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real

size = len(operators)

dimer_corr = np.zeros((size,t_list.shape[0]))
dimer_std = np.zeros((size,t_list.shape[0]))


for s in range(size):
    print(s)
    
    if operators[s]:
        sub2 = operators[s].get_conn_flattened(P_list[0,:,:], sections1)[1].mean().real
        _, mels2_ = operators[s].get_conn_flattened(P_list_, sections2)
        mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        dimer_corr[s] = (mels2 * mels1).mean(axis=1) - 
        dimer_std[s] = (mels2 * mels1).std(axis=1)
    else:
        dimer_corr[s] = 0
        dimer_std[s] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [ ]:
length = [4, 4]
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))


l = []
for i in range(length[0]):
    for j in range(length[1]):
        l.append([i, j])
l = np.array(l)
a = 0
a_ = [a for _ in range(np.prod(length))]
a = np.array(a_)



edges, colors = hex_.dimer_corr(l,a)
operators = f.return_spin_corr(hi, edges, colors)





size = len(operators)
Dimer = np.zeros((size,t_list.shape[0]))
total_num_samples = np.zeros(t_list.shape[0])


P_list = np.load(currentpath+f'/save/dynamics/h=1.0V=0.9l=[4, 4]/P_n=1.0e+04_{i}.npy')

dimer_corr = np.zeros((size,t_list.shape[0]))
dimer_std = np.zeros((size,t_list.shape[0]))


num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])


_, mels1 = operators[0].get_conn_flattened(P_list_, sections2)
mels1 = mels1.reshape(P_list.shape[0], P_list.shape[1]).real
sub1 = mels1[0].mean().real






for s in range(size):
    print(s)
    if operators[s]:

        _, mels2 = operators[s].get_conn_flattened(P_list[0,:,:], sections1)
        mels2 = mels2.real
        sub2 = mels2.mean().real
#         mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        print(mels1.shape, mels2.shape)
        dimer_corr[s] = (mels2 * mels1).mean(axis=1)
        dimer_std[s] = (mels2 * mels1).std(axis=1)
    else:
        dimer_corr[s] = 0
        dimer_std[s] = 0



file_name = 'test_dimer_corr.dat'
f.save_corr(file_name, Dimer, t_list)